##### BIBLIOTECAS

In [15]:
import pandas as pd
import numpy as np
from binance.client import Client
import os
from google.cloud import bigquery
from google.oauth2 import service_account

##### VARIAVEL PARA CONEXÃO COM O GOOGLE BIGQUERY

In [16]:
credentials = service_account.Credentials.from_service_account_file("C:\Development\ETL-Binance\GBQ.json",
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform", "https://www.googleapis.com/auth/drive"])

# EXTRACT

##### CONEXÃO COM A API DA BINANCE

In [17]:
api_key = os.getenv(key = 'BINANCE_API_KEY')
api_secret = os.getenv(key = 'BINANCE_SECRET_KEY')
client = Client(api_key, api_secret)

##### FUNÇÃO PARA UTILIZAÇÃO DA CHAMADA DE API PARA TODOS OS ATIVOS DA BINANCE
##### 1 - MANIPULAÇÃO DA DATA PARA MELHOR VISUALIZAÇÃO
##### 2 - MANIPULAÇÃO DE DATAFRAME - SELECIONANDO COLUNAS RELEVANTES

In [18]:
def binanceDataFrame(symbol):
    ar = np.array(client.get_historical_klines(symbol = symbol,
                             interval = Client.KLINE_INTERVAL_1HOUR,
                             start_str = '1 Jan, 2023'))
    
    df = pd.DataFrame(ar, dtype = str, columns= ("Open_Time",
                                                "Open",
                                                "High",
                                                "Low",
                                                "Close",
                                                "Volume",
                                                "Close_Time",
                                                "Quote asset Volume",
                                                "Number_of_trades",
                                                "Taker buy base asset",
                                                "Taker buy quote asset",
                                                "Ignore"))
    
    df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    df = df[["Open_Time",
    "Open",
     "High",
     "Low",
     "Close",
     "Volume",
     "Close_Time",
     "Number_of_trades"]]
    
    return df

##### EXTRAÇÃO DOS DADOS DO ITEM SELECIONADO PELO SEU SIMBOLO ATRAVÉS DA FUNÇÃO

In [19]:

btc = binanceDataFrame(symbol= 'BTCUSDT')

C:\Users\mateu\AppData\Local\Temp\ipykernel_7904\3366114436.py:19: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
C:\Users\mateu\AppData\Local\Temp\ipykernel_7904\3366114436.py:20: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')


In [20]:
eth = binanceDataFrame(symbol= 'ETHUSDT')

C:\Users\mateu\AppData\Local\Temp\ipykernel_7904\3366114436.py:19: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
C:\Users\mateu\AppData\Local\Temp\ipykernel_7904\3366114436.py:20: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')


# TRANSFORM

##### CRIAÇÃO DE BANDAS DE BOLLINGER

##### MEDIA MOVEL ULTIMAS 20 HORAS

In [21]:

btc['MediaMovel20'] = btc['Close'].shift(1).rolling(20).mean()

In [22]:
eth['MediaMovel20'] = eth['Close'].shift(1).rolling(20).mean()

##### DESVIO PADRÃO ULTIMAS 20 HORAS

In [23]:

btc['DesvioPadrao20'] = btc['Close'].shift(1).rolling(20).std()

In [24]:
eth['DesvioPadrao20'] = eth['Close'].shift(1).rolling(20).std()

##### BANDA INFERIOR DE BOLLINGER

In [25]:

btc['Banda_Inferior'] = btc['MediaMovel20'] - 2*btc["DesvioPadrao20"]
btc['Banda_Superior'] = btc['MediaMovel20'] + 2*btc["DesvioPadrao20"]

In [26]:
eth['Banda_Inferior'] = eth['MediaMovel20'] - 2*eth["DesvioPadrao20"]
eth['Banda_Superior'] = eth['MediaMovel20'] + 2*eth["DesvioPadrao20"]

# LOAD

##### DIRECIONAMENTO DO CSV PARA O GCP - BIGQUERY ATRAVÉS DA CONEXÃO COM A API

In [27]:
btc.to_gbq(credentials=credentials, destination_table='etlBinance.btc', if_exists='replace')

In [28]:
eth.to_gbq(credentials=credentials, destination_table='etlBinance.eth', if_exists='replace')